<a href="https://colab.research.google.com/github/janakurrek/mnli/blob/master/BiLSTM%2BAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import torch
import wget
import os
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

print('Downloading dataset...')

# The URL for the dataset file.
url = 'https://raw.githubusercontent.com/networkdynamics/slur-corpus/main/kurrek.2020.slur-corpus.json'
datapath = './kurrek.2020.slur-corpus.json'

# Download the file (if we haven't already).
if not os.path.exists(datapath):
  wget.download(url, datapath)

corpus = []
with open(datapath) as f:
    for line in f:
        corpus.append(json.loads(line))

corpus = pd.DataFrame(corpus)

# remove CMP classes
corpus.drop(corpus[corpus.gold_label == 'CMP'].index, inplace=True)
corpus['binary_label'] = corpus['gold_label']
corpus['binary_label'] = corpus['binary_label'].replace(['APR', 'HOM'], 'NDG')
corpus['clean_labels'] = pd.factorize(corpus.binary_label)[0]

# split into test, train, validation
from sklearn.model_selection import train_test_split

X = corpus.body
y = corpus.clean_labels

# split test and training set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

datapath_trn, datapath_val, datapath_tst = "./kurrek.2020-slur-corpus-trn.json",\
                                            "./kurrek.2020-slur-corpus-val.json",\
                                            "./kurrek.2020-slur-corpus-tst.json"
corpus.to_json(datapath_trn)
corpus.to_json(datapath_val)
corpus.to_json(datapath_tst)

**Pay "Attention" to Your Context**

https://www.aclweb.org/anthology/W19-3508.pdf

https://github.com/tuhinjubcse/ALW3-ACL2019

https://www.aclweb.org/anthology/N16-1174/

https://github.com/uvipen/Hierarchical-attention-networks-pytorch/blob/master/src/utils.py


*   Stacked Bi-LSTM models outperformed the simple Bi-LSTM models
*   Best performing model on abusive language datasets is the stacked Bi-LSTM with contextual attention



In [14]:
from torchtext import data
from torchtext.vocab import GloVe

max_input_len = 320

# tokenizer settings
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

unk_token_idx = tokenizer.unk_token_id
pad_token_idx = tokenizer.pad_token_id


def tokenize_and_cut(sentence, max_input_length = 320):
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length - 2]
    return tokens

# load inputs and labels
TEXT = data.Field(batch_first=True, use_vocab=False, tokenize=tokenize_and_cut,
                  preprocessing=tokenizer.convert_tokens_to_ids,
                  pad_token=pad_token_idx, unk_token=unk_token_idx)

LABEL = data.Field(
    sequential=False,
    dtype=torch.float
)

# split into training, testing, validation set
D_trn = data.TabularDataset(
    path=datapath_trn, format='json',
    fields={'body': ('text', TEXT),
            'clean_labels': ('labels', LABEL)})

D_val = data.TabularDataset(
    path=datapath_val, format='json',
    fields={'body': ('text', TEXT),
            'clean_labels': ('labels', LABEL)})

D_tst = data.TabularDataset(
    path=datapath_tst, format='json',
    fields={'body': ('text', TEXT),
            'clean_labels': ('labels', LABEL)})

# build vocabulary and count tokens
TEXT.build_vocab(D_trn)
LABEL.build_vocab(D_trn)

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:13: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [16]:
# load fasttext vectors
# TEXT.vocab.load_vectors('glove.twitter.27B.100d')
print(f"Unique tokens in text vocabulary: {len(TEXT.vocab)}")

Unique tokens in text vocabulary: 849


In [17]:
"""Hyperparameters"""
# training
num_epochs = 4
learning_rate = 0.001
batch_size = 128

# model hyperparameters
load_model = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = len(source.vocab)
output_size = len(labels)
embed_size = 50
hidden_size = 1024 # 2014 benchmark; slightly small
num_layers = 2 # benchmark did 4
enc_dropout = 0.5
dec_dropout = 0.5

"""Keon Attention Class"""
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        
        self.hidden_size = hidden_size
        self.attn = nn.Linear(self.hidden_size * 4, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))
        stdv = 1. / math.sqrt(self.v.size(0))
        self.v.data.uniform_(-stdv, stdv)

    def forward(self, hidden, encoder_outputs):
        hidden = hidden.reshape((1, hidden.shape[1], hidden.shape[2] * 2))
        timestep = encoder_outputs.size(0)
        h = hidden.repeat(timestep, 1, 1).transpose(0, 1)
        encoder_outputs = encoder_outputs.transpose(0, 1)  # [B*T*H]
        attn_energies = self.score(h, encoder_outputs)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

    def score(self, hidden, encoder_outputs):
        # [B*T*2H]->[B*T*H]
        catted = torch.cat([hidden, encoder_outputs], 2)
        energy = F.relu(self.attn(catted))
        energy = energy.transpose(1, 2)  # [B*H*T]
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)  # [B*1*H]
        energy = torch.bmm(v, energy)  # [B*1*T]
        return energy.squeeze(1)  # [B*T]

"""Classifier"""
class MultiNLIModel(nn.Module):
    def __init__(self, input_size, output_size, embed_size, device,
                 hidden_size, batch_size, dropout, n_layers, n_cells):
        
        super(MultiNLIModel, self).__init__()
        
        self.device = device
        self.batch_size = batch_size
        self.hidden_size = hidden_size
        self.n_cells = n_cells
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.embed = nn.Embedding(input_size, embed_size)
        self.attention = Attention(hidden_size)
        self.lstm = nn.LSTM(embed_size, hidden_size,
                            num_layers=n_layers, dropout=dropout, 
                            bidirectional=True)
        
        self.fc_hidden = nn.Linear(hidden_size * 2, hidden_size, bias=False)
        self.fc_output = nn.Linear(hidden_size,  output_size, bias=False)
    
    def encode(self, embed):
        # pass embedding input through lstm
        state_shape = self.n_cells, self.batch_size, self.hidden_size
        h0 = c0 = embed.new_zeros(state_shape)
        outputs, (ht, ct) = self.lstm(embed, (h0, c0))

        # pass outcomes through attention layer
        weights = self.attention(ht[-2:], outputs)
        context = weights.bmm(outputs.transpose(0, 1))
        context = context.transpose(0, 1)
        context = context.squeeze(0)
        return context
        
    def forward(self, comment):
        # seq_length, batch_size, embed_size
        comment_embed = self.dropout(self.embed(comment))
        comment_contx = self.encode(comment)
        # seq_len, hidden_size * 2
        comment_contx = self.relu(self.fc_hidden(comment_contx))
        # hidden_size * 2, output_size
        comment_contx = self.relu(self.fc_output(comment_contx))
        return pair_output

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (D_trn, D_val, D_tst), 
     batch_size = batch_size,
     sort_within_batch = True,
     sort_key = lambda x: len(x.text), # minimize padding
     device = device)

encoder = Encoder(input_size, embed_size, hidden_size,
                  n_layers, enc_dropout).to(device)

decoder = Decoder(input_size, embed_size, hidden_size,
                  output_size, n_layers, dec_dropout).to(device)

model = Classifier(encoder, decoder).to(device)

pad_idx_in_vocab = source.vocab.stoi["<pad"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx_in_vocab)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

NameError: ignored